In [1]:
import pandas as pd
import math

In [100]:
df = pd.read_csv("dataset/Aalborg_Tag.csv",sep=";")
df["Recorded"] = pd.to_datetime(df["Recorded"], format="%d-%m-%Y %H:%M:%S")


In [101]:
newdf = df.groupby([df["Recorded"].dt.year,df["Recorded"].dt.month,df["Recorded"].dt.hour,df["Recorded"].dt.minute]).mean()
newdf.index.names = ["year","month","hour","minute"]


In [102]:
emptydf = pd.DataFrame(index=[201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,
                              202107,202108,202109,202110,202111,202112,202201,202202,202203#,202204,202205,202206,202207,202208,202209,202210,202211,202212
                              ])

In [103]:
aqlist = ["NO2","NOx","O3",]
for aq in aqlist:
    for i in range(0,24):
        for j in range(0,60,30):
            emptydf[aq+ "_" + str(i) +":"+ str(j)] = 1

C:\Users\Nobody\AppData\Local\Temp/ipykernel_12496/1648068968.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  emptydf[aq+ "_" + str(i) +":"+ str(j)] = 1
C:\Users\Nobody\AppData\Local\Temp/ipykernel_12496/1648068968.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  emptydf[aq+ "_" + str(i) +":"+ str(j)] = 1
C:\Users\Nobody\AppData\Local\Temp/ipykernel_12496/1648068968.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi

In [104]:
yearmonthhourdf = newdf.copy()
yearmonthhourdf = yearmonthhourdf.reset_index()

In [105]:
indexdict = {0:201912,1:202001,2:202002,3:202003,4:202004,5:202005,6:202006,7:202007,8:202008,9:202009,10:202010,11:202011,12:202012,13:202101,14:202102,15:202103,16:202104,17:202105,18:202106,
                              19:202107,20:202108,21:202109,22:202110,23:202111,24:202112,25:202201,26:202202,27:202203,28:202204,29:202205,30:202206,31:202207,32:202208,33:202209,
                              34:202210,35:202211,36:202212}

for aq in aqlist:
    for j in range(0,28):
        l = -1
        for i in range (0,24):
            for k in range(0,2):
                l += 1
                if aq == "O3":
                    emptydf.loc[indexdict[j],aq+ "_" + str(i) + ":" + str(k*30)] = yearmonthhourdf.loc[l+48*j,"O3"]
                elif aq == "NO2":
                    emptydf.loc[indexdict[j],aq+ "_" + str(i)+ ":" + str(k*30)] = yearmonthhourdf.loc[l+48*j,"NO2"]
                elif aq == "NOx":
                    emptydf.loc[indexdict[j],aq+ "_" + str(i)+ ":" + str(k*30)] = yearmonthhourdf.loc[l+48*j,"NOx"]
                


In [106]:
dftoconvert = df.copy()

In [107]:
dftoconvert.sort_values(by="Recorded",inplace=True)
dftoconvert.index = dftoconvert["Recorded"]

In [108]:
dftoconvert['time_year'] = dftoconvert.index.map(lambda x: x.year)
dftoconvert['time_month'] = dftoconvert.index.map(lambda x: x.month)
dftoconvert['time_day'] = dftoconvert.index.map(lambda x: x.day)
dftoconvert['time_hour'] = dftoconvert.index.map(lambda x: x.hour)
dftoconvert['time_minute'] = dftoconvert.index.map(lambda x: x.minute)

In [109]:
#aqdict = {1:"CO_",2:"NO2_",3:"SO2_",4:"NOx_"}
aqdict = {1:"NO2_",2:"NOx_",3:"O3_"}
#print("aq ,     time,    now,    mean,  counter")
counter = 0
x = 0.999999
for i in range(0,dftoconvert.shape[0]):
    for j in range(1,4):
        for k in range(7,0,-1):
            flooringnumber = math.floor((x*10**k))/10**k
            if dftoconvert.iloc[i,j] > emptydf[aqdict[j]+str(dftoconvert.iloc[i,-2])+":"+str(dftoconvert.iloc[i,-1])].quantile(flooringnumber):
                counter +=1
                #print(aqdict[j],",",dftoconvert.iloc[i,0],",",dftoconvert.iloc[i,j],",",emptydf[aqdict[j]+str(dftoconvert.iloc[i,-2])+":"+str(dftoconvert.iloc[i,-1])].quantile(flooringnumber).round(2),",",str(counter))
                dftoconvert.iloc[i,j] = emptydf[aqdict[j]+str(dftoconvert.iloc[i,-2])+":"+str(dftoconvert.iloc[i,-1])].quantile(flooringnumber)
                break

In [111]:
dftoconvert = dftoconvert.round(2)
dftoconvert= dftoconvert[["Recorded","NO2","NOx","O3"]]
dftoconvert.to_csv("Dataset/AAL_T_19-22mar_Skew.csv",index=False)

In [ ]:
dftoconvert= dftoconvert[[[["Recorded","NO2","NOx","O3"]]]]
desendingdf = dftoconvert.copy()
desendingdf = desendingdf.reset_index(drop=True)
desendingdf = desendingdf.sort_values(by="Recorded",ascending=False)
desendingdf.to_csv("Aalborg_Tag_SkewFix_desending.csv",index=False)